<a href="https://colab.research.google.com/github/robdicoco/solid-robot/blob/main/Google_agent_sample.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install Google GenAI library
%pip -q install google-genai



In [2]:
# Configure the Google Gemini API Key

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('Gemini_API')

In [3]:
# Configure the Gemini SDK client

from google import genai

client = genai.Client()

MODEL_ID = "gemini-2.0-flash"

In [4]:
# Ask Gemini for information more recent than its knowledge base

from IPython.display import HTML, Markdown

#  Ask the model about the next AI immersion ###
response = client.models.generate_content(
    model=MODEL_ID,
    contents='When will be the next imersion of Google Gemini from Alura?'
)


# Display the answer on the screen
display(Markdown(f"Response :\n {response.text}"))

Response :
 I don't have access to a real-time, up-to-the-minute course schedule for Alura. Alura's course offerings change frequently.

The best way to find out about the next "Google Gemini Immersion" course from Alura is to check directly on their website:

*   **Alura's Website:** Go to [https://www.alura.com.br/](https://www.alura.com.br/) and search for "Google Gemini" or "Gemini".  Look for upcoming courses or learning paths.

*   **Alura's Course Calendar:** Look for a course calendar or schedule on their site to see when new courses are being offered.

*   **Alura's Social Media:** Follow Alura on social media (LinkedIn, Twitter, etc.) to get announcements about new courses.

*   **Contact Alura Directly:** If you can't find the information online, contact Alura's customer support and ask them directly about upcoming Google Gemini immersion courses.


In [5]:
# Ask Gemini for information using Google Search as context


response = client.models.generate_content(
    model=MODEL_ID,
    contents='When will be the next imersion of Google Gemini from Alura?',
    config = {"tools": [{"google_search": {}}]}
)

# Display the response
display(Markdown(f"Response:\n {response.text}"))

Response:
 I am unable to find information regarding the next "immersion of Google Gemini from Alura." However, I can share some information regarding Google Gemini and Alura:

*   Alura offered a free "AI Agents Immersion using Google Gemini" course that took place May 12th to 16th, 2025.
*   Google's Gemini 2.0 is available in the Gemini app.
*   Alura is a platform that offers online courses and training in technology and digital skills.



In [6]:
# Display the search query
print(f"Search performed: {response.candidates[0].grounding_metadata.web_search_queries}")
# Display the URLs used as basis for the response
print(f"Pages used in the response: {', '.join([site.web.title for site in response.candidates[0].grounding_metadata.grounding_chunks])}")
print()
display(HTML(response.candidates[0].grounding_metadata.search_entry_point.rendered_content))

Search performed: ['Alura Google Gemini immersion next date']
Pages used in the response: reddit.com, blog.google



In [7]:
# Install Google ADK Agents Framework #
%pip install -q google-adk


In [8]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # To create content (Content and Part)
from datetime import date
import textwrap # To better format text output
from IPython.display import display, Markdown  # To make HTTP requests
import warnings

warnings.filterwarnings("ignore")

In [9]:
# Helper function that sends a message to an agent via Runner and returns the final response
def call_agent(agent: Agent, message_text: str) -> str:
    # Creates an in-memory session service
    session_service = InMemorySessionService()
    # Creates a new session (you can customize IDs as needed)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Creates a Runner for the agent
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Creates the input message content
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Asynchronously iterates through the events returned during agent execution
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [10]:
# Helper function to display formatted text in Markdown in Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [11]:
##########################################
# --- Agent 1: News Finder           --- #
##########################################
def agent_news_finder(topic, today_date):
    news_finder = Agent(
        name="agent_news_finder",
        model="gemini-2.0-flash",
        instruction="""
          You are a news research expert.
          Your task is to find up to three of the most recent and highly relevant news articles on the given topic using (google_search).
          Focus on finding reputable news sources published within the last week.
          If the initial topic yields few results, try one closely related alternative.
          Present the findings as a bulleted list with the title and a brief summary of each article.
        """,
        description="Agent that searches for news on a given topic.",
        tools=[google_search]
    )

    input_agent_news_finder = f"Topic: {topic}\nCurrent date: {today_date}"
    # Execute the agent
    news_found = call_agent(news_finder, input_agent_news_finder)
    return news_found



In [12]:
################################################
# --- Agent 2: Post Planner                --- #
################################################
def agent_post_planner(topic, news_found):
    agent_planner = Agent(
        name="agent_post_planner",
        model="gemini-2.0-flash",
        instruction="""
            You are a post planner expert in social network.
            Your task is to create a post plan for Instagram based on the news found.
            You can use (google_search) to cleate a plan about the most important topics.
            You can use (google_search) for further refine the post on the most relevant tópics
           in the end, you will choose the most relevant topic among all of them, based on your
           research and return to this topic, its most relevant points, and the plan with the
           subjects to be addressed in the post that will be written there later.
        """,
        description="Agent that plans post",
        tools=[google_search]
    )


    input_agent_planner = f"Topic: {topic}\nNews found: {news_found}"
    # Execute the agent
    post_plan = call_agent(agent_planner, input_agent_planner)
    return post_plan


In [13]:

######################################
# --- Agent 3: Post Writer       --- #
######################################
def agent_writer(topic, post_plan):
    writer = Agent(
        name="agent_writer",
        model="gemini-2.0-flash",
        instruction="""
            You are a Creative Writer specializing in creating viral posts for social media.
            You write posts for RobDC a Web3 developer, with MSc in Biomedical Engineering, BSc. in Systems Analysis, Mechatronics Tech.
            Use the theme provided in the post plan and the most relevant points provided and, based on that,
            write a draft Instagram post on the indicated topic.
            The post should be engaging, informative, use simple language, and include 2 to 5 hashtags at the end.
            """,
        description="Agent writer of engaging posts for Instagram"
    )

    input_agent_writer = f"Topic: {topic}\nPost plan: {post_plan}"
    # Execute the agent
    draft = call_agent(writer, input_agent_writer)
    return draft


In [14]:
##########################################
# --- Agent 4:     Quality Reviewer --- #
##########################################
def agent_reviewer(topic, draft_information):
    reviewer = Agent(
        name="agent_reviewer",
        model="gemini-2.0-flash",
        instruction="""
            You are a meticulous Content Editor and Reviewer, specializing in social media posts, with a focus on Instagram.
            Since you have a young audience, between 18 and 30 years old, use an appropriate writing tone.
            Review the Instagram post draft below on the indicated topic, checking for clarity, conciseness, correctness, and tone.
            If the draft is good, reply only 'The draft is great and ready to publish!'.
            If there are problems, point them out and suggest improvements.
            """,
        description="Agent reviewer of social media posts"
    )

    input_agent_reviewer = f"Topic: {topic}\nDraft: {draft_information}"
    # Execute the agent
    reviewed_text = call_agent(reviewer, input_agent_reviewer)
    return reviewed_text


In [16]:
today_date = date.today().strftime("%d/%m/%Y")

print("🚀 Starting the Instagram Post Creation System with 4 Agents 🚀🚀🚀")

# --- Get Topic from User ---
topic = input("❓ Please enter the TOPIC you want to create the trend post about: ")

# Insert agent system logic ################################################

if not topic:
    print("❌ Topic cannot be empty. Please try again.")
else:
    print(f"🔍 Searching for news on the topic: {topic}...")
    news_found = agent_news_finder(topic, today_date)
    display("📝 News found:\n", to_markdown(news_found))

    print("📋 Creating post plan...")
    post_plan = agent_post_planner(topic, news_found)
    display("📋 Post plan:\n", to_markdown(post_plan))

    print("✍️ Writing post draft...")
    draft = agent_writer(topic, post_plan)
    display("✍️ Post draft:\n", to_markdown(draft))

    print("🔍 Reviewing post draft...")
    reviewed_text = agent_reviewer(topic, draft)
    display("🔍 Reviewed post:\n", to_markdown(reviewed_text))


🚀 Starting the Instagram Post Creation System with 4 Agents 🚀🚀🚀
❓ Please enter the TOPIC you want to create the trend post about: torradas
🔍 Searching for news on the topic: torradas...


'📝 News found:\n'

> Okay, I'll find some recent news articles about "torradas".
> 
> 
> The search results seem to be mentioning "Torradas" as a name, rather than the food item "toast/torrada". However, there is one article related to a Carnival celebration in Ibiza that mentions "torradas". Here's a summary of that article:
> 
> *   **Torradas And Chirigotas Spice Up Ibiza's Carnival Celebration**: This article discusses the Carnival celebration in Ibiza, which includes "torradas" and chirigotas. The Ibiza Town Hall invested approximately 50,000 euros in the Carnival of 2024, which included a children's costume contest and free "tortilla" for 700 people. The Carnival parade was inclusive, with reserved seating for senior citizens and people with special needs.


📋 Creating post plan...


'📋 Post plan:\n'

...

'✍️ Post draft:\n'

> Okay, I will generate the draft Instagram posts based on the post plan:
> 
> **Post 3: (Image/Video - Food)**
> 
> \[Imagine a vibrant photo or video of "Torradas" (grilled bread) being prepared on a grill or served with delicious toppings at the Vara de Rey tent in Ibiza]
> 
> Fuel up for Carnival in Ibiza! 🔥🤤 Enjoy FREE "torradas" (crispy grilled bread) and other tasty treats at the Vara de Rey tent. It's a true taste of local flavor that will keep you energized for all the festivities! What's your favorite Carnival snack? Tell me in the comments! 👇 #[CarnivalIbiza] #[IbizaFood] #Torradas #[Carnival] #[Foodie]
> 


🔍 Reviewing post draft...


'🔍 Reviewed post:\n'

> The draft is great but needs some improvements.
> 
> Here's what I suggest:
> 
> *   **Clarity and Conciseness:** The text is good, but we can make it even more concise and engaging for the young audience.
> *   **Hashtags:** The hashtags are relevant, but let's add some more specific and trending ones to increase visibility.
> *   **Call to action:** It's good, but we can make it more interactive and specific.
> *   **Emojis:** Let's add more emojis to make the post more appealing and fun.
> 
> Here's the improved draft:
> 
> **Post 3: (Image/Video - Food)**
> 
> \[Imagine a vibrant photo or video of "Torradas" (grilled bread) being prepared on a grill or served with delicious toppings at the Vara de Rey tent in Ibiza]
> 
> Fuel up for Carnival in Ibiza with FREE "torradas"! 🔥🤤 Nothing beats crispy grilled bread to keep you energized for all the festivities at Vara de Rey tent. What toppings would you choose for your torrada? Share your creations in the comments! 👇 #[CarnivalIbiza] #[IbizaEats] #Torradas #[CarnivalVibes] #[StreetFood] #[FoodLovers] #[IbizaParty]
